## Rotten Tomatoes Critic Reviews Data Processing

### Import Libraries

In [16]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize.casual import casual_tokenize
import ast
import requests

In [3]:
df = pd.DataFrame(pd.read_csv('../data/rotten_tomatoes_critic_reviews.csv'))
df.head()

,rotten_tomatoes_link,critic_name,top_critic,publisher_name,review_type,review_score,review_date,review_content
0,m/0814255,Andrew L. Urban,False,Urban Cinefile,Fresh,NaN,2010-02-06,A fantasy adventure that fuses Greek mythology...
1,m/0814255,Louise Keller,False,Urban Cinefile,Fresh,NaN,2010-02-06,"Uma Thurman as Medusa, the gorgon with a coiff..."
2,m/0814255,NaN,False,FILMINK (Australia),Fresh,NaN,2010-02-09,With a top-notch cast and dazzling special eff...
3,m/0814255,Ben McEachen,False,Sunday Mail (Australia),Fresh,3.5/5,2010-02-09,Whether audiences will get behind The Lightnin...
4,m/0814255,Ethan Alter,True,Hollywood Reporter,Rotten,NaN,2010-02-10,What's really lacking in The Lightning Thief i...


In [4]:
df.dtypes

rotten_tomatoes_link    object
critic_name             object
top_critic                bool
publisher_name          object
review_type             object
review_score            object
review_date             object
review_content          object
dtype: object

In [6]:
df.shape

(1130017, 8)

### Data Cleaning

In [5]:
df.isnull().sum()

rotten_tomatoes_link         0
critic_name              18529
top_critic                   0
publisher_name               0
review_type                  0
review_score            305936
review_date                  0
review_content           65806
dtype: int64

`critic_name` is not very useful to the project as we do not need to know who wrote the review, just the sentiment. <br>
`review_content`, while important, is not a must to have, and we can fill it in with blank values.

In [11]:
df['review_content'] = df['review_content'].fillna('No review')

In [9]:
df['review_score'].value_counts()

review_score
3/5        90273
4/5        83659
3/4        72366
2/5        60174
2/4        47546
           ...  
69/70          1
2.5/20         1
5.55/10        1
35             1
9.56/10        1
Name: count, Length: 814, dtype: int64

In [10]:
len(df['review_score'].value_counts())

814

There are 814 unique scores, which is bizzare. <br>
We cannot remove all the rows as it contributes to a significant portion of the data (Roughly 30%). Thus, we cannot use this data.

In [13]:
df2 = df[['review_type', 'review_date', 'review_content']]
df2.head()

,review_type,review_date,review_content
0,Fresh,2010-02-06,A fantasy adventure that fuses Greek mythology...
1,Fresh,2010-02-06,"Uma Thurman as Medusa, the gorgon with a coiff..."
2,Fresh,2010-02-09,With a top-notch cast and dazzling special eff...
3,Fresh,2010-02-09,Whether audiences will get behind The Lightnin...
4,Rotten,2010-02-10,What's really lacking in The Lightning Thief i...


In [25]:
df2['review_date'] = pd.to_datetime(df2['review_date'])
type(df2['review_date'][0])

/var/folders/1v/dvtq9m8d48z659z8qvy5rxdh0000gn/T/ipykernel_33812/1202700846.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['review_date'] = pd.to_datetime(df2['review_date'])


pandas._libs.tslibs.timestamps.Timestamp

In [14]:
nltk.download('punkt', download_dir='./nltk_data', quiet=True)
nltk.download('punkt_tab', download_dir='./nltk_data', quiet=True)
nltk.download('stopwords', download_dir='./nltk_data', quiet=True)
nltk.download('wordnet', download_dir='./nltk_data', quiet=True)
nltk.data.path.append('./nltk_data')

In [17]:
# stop_words = set(stopwords.words('english'))
# stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
stopwords_list = requests.get("https://gist.githubusercontent.com/rg089/35e00abf8941d72d419224cfd5b5925d/raw/12d899b70156fd0041fa9778d657330b024b959c/stopwords.txt").content
stopwords = set(stopwords_list.decode().splitlines())

In [18]:
df2['tokenized_review_content'] = df2['review_content'].apply(lambda x: casual_tokenize(x.lower(), reduce_len=True, strip_handles=False))
df2.head()

/var/folders/1v/dvtq9m8d48z659z8qvy5rxdh0000gn/T/ipykernel_33812/2533644540.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['tokenized_review_content'] = df2['review_content'].apply(lambda x: casual_tokenize(x.lower(), reduce_len=True, strip_handles=False))


,review_type,review_date,review_content,tokenized_review_content
0,Fresh,2010-02-06,A fantasy adventure that fuses Greek mythology...,"[a, fantasy, adventure, that, fuses, greek, my..."
1,Fresh,2010-02-06,"Uma Thurman as Medusa, the gorgon with a coiff...","[uma, thurman, as, medusa, ,, the, gorgon, wit..."
2,Fresh,2010-02-09,With a top-notch cast and dazzling special eff...,"[with, a, top-notch, cast, and, dazzling, spec..."
3,Fresh,2010-02-09,Whether audiences will get behind The Lightnin...,"[whether, audiences, will, get, behind, the, l..."
4,Rotten,2010-02-10,What's really lacking in The Lightning Thief i...,"[what's, really, lacking, in, the, lightning, ..."


In [19]:
df2['alphanum_review_content'] = df2['tokenized_review_content'].apply(lambda x: [word for word in x if word.isalnum()])
df2.head()

/var/folders/1v/dvtq9m8d48z659z8qvy5rxdh0000gn/T/ipykernel_33812/1022228259.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['alphanum_review_content'] = df2['tokenized_review_content'].apply(lambda x: [word for word in x if word.isalnum()])


,review_type,review_date,review_content,tokenized_review_content,alphanum_review_content
0,Fresh,2010-02-06,A fantasy adventure that fuses Greek mythology...,"[a, fantasy, adventure, that, fuses, greek, my...","[a, fantasy, adventure, that, fuses, greek, my..."
1,Fresh,2010-02-06,"Uma Thurman as Medusa, the gorgon with a coiff...","[uma, thurman, as, medusa, ,, the, gorgon, wit...","[uma, thurman, as, medusa, the, gorgon, with, ..."
2,Fresh,2010-02-09,With a top-notch cast and dazzling special eff...,"[with, a, top-notch, cast, and, dazzling, spec...","[with, a, cast, and, dazzling, special, effect..."
3,Fresh,2010-02-09,Whether audiences will get behind The Lightnin...,"[whether, audiences, will, get, behind, the, l...","[whether, audiences, will, get, behind, the, l..."
4,Rotten,2010-02-10,What's really lacking in The Lightning Thief i...,"[what's, really, lacking, in, the, lightning, ...","[really, lacking, in, the, lightning, thief, i..."


In [20]:
df2['stopword_removed_review_content'] = df2['alphanum_review_content'].apply(lambda x: [word for word in x if word not in stopwords])
df2.head()

/var/folders/1v/dvtq9m8d48z659z8qvy5rxdh0000gn/T/ipykernel_33812/337655852.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['stopword_removed_review_content'] = df2['alphanum_review_content'].apply(lambda x: [word for word in x if word not in stopwords])


,review_type,review_date,review_content,tokenized_review_content,alphanum_review_content,stopword_removed_review_content
0,Fresh,2010-02-06,A fantasy adventure that fuses Greek mythology...,"[a, fantasy, adventure, that, fuses, greek, my...","[a, fantasy, adventure, that, fuses, greek, my...","[fantasy, adventure, fuses, greek, mythology, ..."
1,Fresh,2010-02-06,"Uma Thurman as Medusa, the gorgon with a coiff...","[uma, thurman, as, medusa, ,, the, gorgon, wit...","[uma, thurman, as, medusa, the, gorgon, with, ...","[uma, thurman, medusa, gorgon, coiffure, writh..."
2,Fresh,2010-02-09,With a top-notch cast and dazzling special eff...,"[with, a, top-notch, cast, and, dazzling, spec...","[with, a, cast, and, dazzling, special, effect...","[cast, dazzling, special, effects, tide, teens..."
3,Fresh,2010-02-09,Whether audiences will get behind The Lightnin...,"[whether, audiences, will, get, behind, the, l...","[whether, audiences, will, get, behind, the, l...","[audiences, lightning, thief, hard, predict, e..."
4,Rotten,2010-02-10,What's really lacking in The Lightning Thief i...,"[what's, really, lacking, in, the, lightning, ...","[really, lacking, in, the, lightning, thief, i...","[lacking, lightning, thief, genuine, sense, th..."


In [22]:
df2['lemmatized_review_content'] = df2['stopword_removed_review_content'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
df2.head()

/var/folders/1v/dvtq9m8d48z659z8qvy5rxdh0000gn/T/ipykernel_33812/4156547016.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['lemmatized_review_content'] = df2['stopword_removed_review_content'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])


,review_type,review_date,review_content,tokenized_review_content,alphanum_review_content,stopword_removed_review_content,lemmatized_review_content
0,Fresh,2010-02-06,A fantasy adventure that fuses Greek mythology...,"[a, fantasy, adventure, that, fuses, greek, my...","[a, fantasy, adventure, that, fuses, greek, my...","[fantasy, adventure, fuses, greek, mythology, ...","[fantasy, adventure, fuse, greek, mythology, c..."
1,Fresh,2010-02-06,"Uma Thurman as Medusa, the gorgon with a coiff...","[uma, thurman, as, medusa, ,, the, gorgon, wit...","[uma, thurman, as, medusa, the, gorgon, with, ...","[uma, thurman, medusa, gorgon, coiffure, writh...","[uma, thurman, medusa, gorgon, coiffure, writh..."
2,Fresh,2010-02-09,With a top-notch cast and dazzling special eff...,"[with, a, top-notch, cast, and, dazzling, spec...","[with, a, cast, and, dazzling, special, effect...","[cast, dazzling, special, effects, tide, teens...","[cast, dazzling, special, effect, tide, teen, ..."
3,Fresh,2010-02-09,Whether audiences will get behind The Lightnin...,"[whether, audiences, will, get, behind, the, l...","[whether, audiences, will, get, behind, the, l...","[audiences, lightning, thief, hard, predict, e...","[audience, lightning, thief, hard, predict, en..."
4,Rotten,2010-02-10,What's really lacking in The Lightning Thief i...,"[what's, really, lacking, in, the, lightning, ...","[really, lacking, in, the, lightning, thief, i...","[lacking, lightning, thief, genuine, sense, th...","[lacking, lightning, thief, genuine, sense, th..."


In [26]:
df2.to_csv('../data/rotten_tomatoes_critic_reviews_cleaned.csv', index=False)